In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.6 MB/s eta 0:00:00


In [ ]:
#preprocessing the data
import json
import os
import pandas as pd

with open('/Users/baharulislam/Desktop/DATA_SCIENCE/LLM/SEMEVAL/TASK 2 2024/nli4ct-main/Complete_dataset/train.json') as f:
    train_data = json.load(f)

clinical_trials = {}
ct_folder = 'Complete_dataset/CT json'
for filename in os.listdir(ct_folder):
    if filename.endswith('.json'):
        with open(os.path.join(ct_folder, filename)) as f:
            trial_data = json.load(f)
            clinical_trials[trial_data["Clinical Trial ID"]] = trial_data

def get_full_section_details(trial_data, section_id):
    return " ".join(trial_data.get(section_id, []))

def get_indexed_section_details(trial_data, section_id, indexes):
    section_details = trial_data.get(section_id, [])
    return " ".join([section_details[i] for i in indexes if i < len(section_details)])

merged_data = []

for record_id, record in train_data.items():
    primary_id = record["Primary_id"]
    secondary_id = record.get("Secondary_id")

    primary_trial = clinical_trials.get(primary_id, {})
    primary_evidence = get_indexed_section_details(primary_trial, record["Section_id"], record["Primary_evidence_index"])
    primary_full_section = get_full_section_details(primary_trial, record["Section_id"])

    if secondary_id:
        secondary_trial = clinical_trials.get(secondary_id, {})
        secondary_evidence = get_indexed_section_details(secondary_trial, record["Section_id"], record.get("Secondary_evidence_index", []))
        secondary_full_section = get_full_section_details(secondary_trial, record["Section_id"])
    else:
        secondary_evidence = None
        secondary_full_section = None

    reference_data = primary_full_section
    if secondary_full_section:
        reference_data += " " + secondary_full_section

    merged_record = {
        "test_ID": record_id,
        "Type": record["Type"],
        "Section_id": record["Section_id"],
        "Primary_id": primary_id,
        "Secondary_id": secondary_id,
        "Statement": record["Statement"],
        "Label": record["Label"],
        "Primary_evidence_index": primary_evidence,
        "Secondary_evidence_index": secondary_evidence,
        "Reference_data": reference_data
    }

    merged_data.append(merged_record)

df = pd.DataFrame(merged_data)
df.to_csv('merged_dataset_new.csv', index=False)

print("CSV file created successfully.")


In [2]:
import openai
import pandas as pd

In [3]:
df = pd.read_csv('merged_dataset_new.csv')

In [4]:
df.head()

,test_ID,Type,Section_id,Primary_id,Secondary_id,Statement,Label,Primary_evidence_index,Secondary_evidence_index,Reference_data
0,5bc844fc-e852-4270-bfaf-36ea9eface3d,Comparison,Intervention,NCT01928186,NCT00684983,All the primary trial participants do not rece...,Contradiction,INTERVENTION 1: Diagnostic (FLT PET) Pati...,INTERVENTION 1: Arm A Patients receive or...,INTERVENTION 1: Diagnostic (FLT PET) Pati...
1,86b7cb3d-6186-4a04-9aa6-b174ab764eed,Single,Eligibility,NCT00662129,NaN,"Patients with Platelet count over 100,000/mm¬¨...",Contradiction,"PATIENT CHARACTERISTICS: ANC 1,500/mm³ ...",NaN,DISEASE CHARACTERISTICS: Histologically or c...
2,dbed5471-c2fc-45b5-b26f-430c9fa37a37,Comparison,Adverse Events,NCT00093145,NCT00703326,Heart-related adverse events were recorded in ...,Entailment,Adverse Events 1: Supraventricular tachycard...,Adverse Events 1: Atrial fibrillation 1/752 ...,Adverse Events 1: Total: 5/32 (15.63%) Feb...
3,20c35c89-8d23-4be3-b603-ac0ee0f3b4de,Single,Eligibility,NCT01097642,NaN,Adult Patients with histologic confirmation of...,Contradiction,Inclusion Criteria: Patients with histologic...,NaN,Inclusion Criteria: Patients with histologic...
4,f17cb242-419d-4f5d-bfa4-41494ed5ac0e,Comparison,Intervention,NCT00852930,NCT02308020,Laser Therapy is in each cohort of the primary...,Contradiction,INTERVENTION 1: Laser Therapy Alone thera...,"INTERVENTION 1: Part A Abemaciclib: HR+, HE...",INTERVENTION 1: Laser Therapy Alone thera...


In [9]:
df.columns

Index(['test_ID', 'Type', 'Section_id', 'Primary_id', 'Secondary_id',
       'Statement', 'Label', 'Primary_evidence_index',
       'Secondary_evidence_index', 'Reference_data'],
      dtype='object')

In [13]:
df['Primary_evidence_index'][0]

'INTERVENTION 1:    Diagnostic (FLT PET)   Patients with early stage, ER positive primary breast cancer undergo FLT PET scan at baseline and 1-6 weeks after the start of standard endocrine treatment. The surgery follows 1-7 days after the second FLT PET scan.   Tracer used in the FLT PET (positron emission tomography) scanning procedure: [F18] fluorothymidine.   Positron Emission Tomography: Undergo FLT PET   Laboratory Biomarker Analysis: Correlative studies - Ki67 staining of the tumor tissue in the biopsy and surgical specimen.'

In [14]:
df['Reference_data'][0]

'INTERVENTION 1:    Diagnostic (FLT PET)   Patients with early stage, ER positive primary breast cancer undergo FLT PET scan at baseline and 1-6 weeks after the start of standard endocrine treatment. The surgery follows 1-7 days after the second FLT PET scan.   Tracer used in the FLT PET (positron emission tomography) scanning procedure: [F18] fluorothymidine.   Positron Emission Tomography: Undergo FLT PET   Laboratory Biomarker Analysis: Correlative studies - Ki67 staining of the tumor tissue in the biopsy and surgical specimen. INTERVENTION 1:    Arm A   Patients receive oral capecitabine twice daily on days 1-14 and oral lapatinib ditosylate once daily on days 1-21. Courses repeat every 21 days in the absence of disease progression or unacceptable toxicity. lapatinib ditosylate: Given PO and capecitabine: Given PO INTERVENTION 2:    Arm B   Patients receive capecitabine and lapatinib ditosylate as in arm I. Patients also receive cixutumumab IV over 1-1½ hours on days 1, 8, and 15. 

In [15]:
df['Secondary_evidence_index'][0]

'INTERVENTION 1:    Arm A   Patients receive oral capecitabine twice daily on days 1-14 and oral lapatinib ditosylate once daily on days 1-21. Courses repeat every 21 days in the absence of disease progression or unacceptable toxicity. lapatinib ditosylate: Given PO and capecitabine: Given PO INTERVENTION 2:    Arm B   Patients receive capecitabine and lapatinib ditosylate as in arm I. Patients also receive cixutumumab IV over 1-1½ hours on days 1, 8, and 15. Courses repeat every 21 days in the absence of disease progression or unacceptable toxicity. cixutumumab: Given IV, lapatinib ditosylate: Given PO and capecitabine: Given PO'

In [16]:
df['Primary_evidence_index'][1]

'  PATIENT CHARACTERISTICS:   ANC  1,500/mm³   Platelet count  100,000/mm³   Hemoglobin  9.0 g/dL'

In [17]:
df['Secondary_evidence_index'][1]

nan

In [18]:
df['Reference_data'][1]

'DISEASE CHARACTERISTICS:   Histologically or cytologically confirmed infiltrating breast cancer   Clinical evidence of metastatic disease   Measurable disease, defined as at least one measurable lesion per RECIST criteria   No non-measurable disease only, defined as all other lesions, including small lesions (longest diameter < 2 cm) and truly non-measurable lesions, including any of the following:   Bone lesions   Leptomeningeal disease   Ascites   Pleural/pericardial effusion   Inflammatory breast disease   Lymphangitis cutis/pulmonis   Abdominal masses that are not confirmed and followed by imaging techniques   Cystic lesions   Patients with HER-2/neu positive tumors, must have received prior treatment with trastuzumab (Herceptin®) or have a contraindication for trastuzumab   No evidence of active brain metastasis, including leptomeningeal involvement, on MRI or CT scan   CNS metastasis controlled by prior surgery and/or radiotherapy allowed   Must be asymptomatic for  2 months wit

In [20]:
df['Statement'][1]

'Patients with Platelet count over 100,000/mm¬¨‚â•, ANC <  1,700/mm¬¨‚â• and Hemoglobin between 4 to 5 grams per deciliter are eligible for the primary trial.'

In [21]:
df['Label'][1]

'Contradiction'

In [5]:
df.shape

(1700, 10)

In [6]:
df = df[0:700]

In [7]:
df['Response_strings']= df.apply(lambda row: f"""Label: {row['Label']}
Primary Evidence Indexes = {row['Primary_evidence_index']}
Secondary Evidence Indexes = {row['Secondary_evidence_index']}
""", axis = 1)

In [8]:
df['Input'] = "Type: "+ df['Type']+ "\nStatement: "+ df['Statement']+ "\nReference Data: " + df['Reference_data']

In [9]:
df.head()

,test_ID,Type,Section_id,Primary_id,Secondary_id,Statement,Label,Primary_evidence_index,Secondary_evidence_index,Reference_data,Response_strings,Input
0,5bc844fc-e852-4270-bfaf-36ea9eface3d,Comparison,Intervention,NCT01928186,NCT00684983,All the primary trial participants do not rece...,Contradiction,INTERVENTION 1: Diagnostic (FLT PET) Pati...,INTERVENTION 1: Arm A Patients receive or...,INTERVENTION 1: Diagnostic (FLT PET) Pati...,Label: Contradiction\nPrimary Evidence Indexes...,Type: Comparison\nStatement: All the primary t...
1,86b7cb3d-6186-4a04-9aa6-b174ab764eed,Single,Eligibility,NCT00662129,NaN,"Patients with Platelet count over 100,000/mm¬¨...",Contradiction,"PATIENT CHARACTERISTICS: ANC 1,500/mm³ ...",NaN,DISEASE CHARACTERISTICS: Histologically or c...,Label: Contradiction\nPrimary Evidence Indexes...,Type: Single\nStatement: Patients with Platele...
2,dbed5471-c2fc-45b5-b26f-430c9fa37a37,Comparison,Adverse Events,NCT00093145,NCT00703326,Heart-related adverse events were recorded in ...,Entailment,Adverse Events 1: Supraventricular tachycard...,Adverse Events 1: Atrial fibrillation 1/752 ...,Adverse Events 1: Total: 5/32 (15.63%) Feb...,Label: Entailment\nPrimary Evidence Indexes = ...,Type: Comparison\nStatement: Heart-related adv...
3,20c35c89-8d23-4be3-b603-ac0ee0f3b4de,Single,Eligibility,NCT01097642,NaN,Adult Patients with histologic confirmation of...,Contradiction,Inclusion Criteria: Patients with histologic...,NaN,Inclusion Criteria: Patients with histologic...,Label: Contradiction\nPrimary Evidence Indexes...,Type: Single\nStatement: Adult Patients with h...
4,f17cb242-419d-4f5d-bfa4-41494ed5ac0e,Comparison,Intervention,NCT00852930,NCT02308020,Laser Therapy is in each cohort of the primary...,Contradiction,INTERVENTION 1: Laser Therapy Alone thera...,"INTERVENTION 1: Part A Abemaciclib: HR+, HE...",INTERVENTION 1: Laser Therapy Alone thera...,Label: Contradiction\nPrimary Evidence Indexes...,Type: Comparison\nStatement: Laser Therapy is ...


In [10]:
df['Response_strings'][0]

'Label: Contradiction\nPrimary Evidence Indexes = INTERVENTION 1:    Diagnostic (FLT PET)   Patients with early stage, ER positive primary breast cancer undergo FLT PET scan at baseline and 1-6 weeks after the start of standard endocrine treatment. The surgery follows 1-7 days after the second FLT PET scan.   Tracer used in the FLT PET (positron emission tomography) scanning procedure: [F18] fluorothymidine.   Positron Emission Tomography: Undergo FLT PET   Laboratory Biomarker Analysis: Correlative studies - Ki67 staining of the tumor tissue in the biopsy and surgical specimen.\nSecondary Evidence Indexes = INTERVENTION 1:    Arm A   Patients receive oral capecitabine twice daily on days 1-14 and oral lapatinib ditosylate once daily on days 1-21. Courses repeat every 21 days in the absence of disease progression or unacceptable toxicity. lapatinib ditosylate: Given PO and capecitabine: Given PO INTERVENTION 2:    Arm B   Patients receive capecitabine and lapatinib ditosylate as in arm

In [15]:
df['Input'][0]

'Type: Comparison\nStatement: All the primary trial participants do not receive any oral capecitabine, oral lapatinib ditosylate or cixutumumab IV, in conrast all the secondary trial subjects receive these.\nReference Data: INTERVENTION 1:    Diagnostic (FLT PET)   Patients with early stage, ER positive primary breast cancer undergo FLT PET scan at baseline and 1-6 weeks after the start of standard endocrine treatment. The surgery follows 1-7 days after the second FLT PET scan.   Tracer used in the FLT PET (positron emission tomography) scanning procedure: [F18] fluorothymidine.   Positron Emission Tomography: Undergo FLT PET   Laboratory Biomarker Analysis: Correlative studies - Ki67 staining of the tumor tissue in the biopsy and surgical specimen. INTERVENTION 1:    Arm A   Patients receive oral capecitabine twice daily on days 1-14 and oral lapatinib ditosylate once daily on days 1-21. Courses repeat every 21 days in the absence of disease progression or unacceptable toxicity. lapat

In [11]:
system_prompt = "You are an advanced medical reviewer. I will provide type, statement and reference data. Your task is to determine whether the statement is an entailment or a contradiction based on the provided reference. Select evidence from the reference to support your chosen label."

In [12]:
all_conversations = []
for idx, row in df.iterrows():
    all_conversations.append({"messages": [{"role":"system", "content": system_prompt},
    {"role":"user", "content": row['Input']},
    {"role":"assistant", "content": row['Response_strings']}]})


In [13]:
all_conversations[0]

{'messages': [{'role': 'system',
   'content': 'You are an advanced medical reviewer. I will provide type, statement and reference data. Your task is to determine whether the statement is an entailment or a contradiction based on the provided reference. Select evidence from the reference to support your chosen label.'},
  {'role': 'user',
   'content': 'Type: Comparison\nStatement: All the primary trial participants do not receive any oral capecitabine, oral lapatinib ditosylate or cixutumumab IV, in conrast all the secondary trial subjects receive these.\nReference Data: INTERVENTION 1:    Diagnostic (FLT PET)   Patients with early stage, ER positive primary breast cancer undergo FLT PET scan at baseline and 1-6 weeks after the start of standard endocrine treatment. The surgery follows 1-7 days after the second FLT PET scan.   Tracer used in the FLT PET (positron emission tomography) scanning procedure: [F18] fluorothymidine.   Positron Emission Tomography: Undergo FLT PET   Laborator

In [14]:
import json
with open("instances.jsonl", "w") as f:
    for item in all_conversations:
        json.dump(item, f)
        f.write('\n')

In [15]:
client = openai.OpenAI(api_key = "sk-proj-yoc8joQarRMDAf3UtZE1T3BlbkFJG71i8B5VBT6IvEB815Uz")

In [45]:
with open('instances.jsonl', 'rb') as f:
    response = client.files.create(file= f, purpose = 'fine-tune')

In [46]:
response

FileObject(id='file-jUaDUw29DKHAUCXQdGJNRUn6', bytes=2141843, created_at=1720500064, filename='instances.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [16]:
file_id = "file-jUaDUw29DKHAUCXQdGJNRUn6"

In [48]:
response = client.fine_tuning.jobs.create(
    training_file=file_id,
    model = "gpt-3.5-turbo"
)

In [18]:
job_id = "ftjob-pKYeiWBlkquWZcvsqYnGvEmt"

In [19]:
client.fine_tuning.jobs.list(limit=1)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-pKYeiWBlkquWZcvsqYnGvEmt', created_at=1720500097, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::9iyNIzGd', finished_at=1720505538, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-fm4pb04YgBJCbEf9e4EMqlFM', result_files=['file-5eZeeuzl78KWzatEwub0UEqJ'], seed=1851155288, status='succeeded', trained_tokens=1607889, training_file='file-jUaDUw29DKHAUCXQdGJNRUn6', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)], object='list', has_more=True)

In [20]:
df2 = pd.read_csv('merged_dataset_new.csv')

In [21]:
df2 = df2[701:]

In [22]:
df2.shape

(999, 10)

In [23]:
df2['Response_strings']= df2.apply(lambda row: f"""Label: {row['Label']}
Primary Evidence Indexes = {row['Primary_evidence_index']}
Secondary Evidence Indexes = {row['Secondary_evidence_index']}
""", axis = 1)

In [24]:
df2['Input'] = "Type: "+ df2['Type']+ "\nStatement: "+ df2['Statement']+ "\nReference Data: " + df2['Reference_data']

In [25]:
df2.head()

,test_ID,Type,Section_id,Primary_id,Secondary_id,Statement,Label,Primary_evidence_index,Secondary_evidence_index,Reference_data,Response_strings,Input
701,4e31e4f8-a6da-49fc-8bfe-1309674ec2ee,Single,Results,NCT00244881,NaN,30% of the primary trial participants had an i...,Contradiction,Outcome Measurement: Fraction of Patients W...,NaN,Outcome Measurement: Fraction of Patients W...,Label: Contradiction\nPrimary Evidence Indexes...,Type: Single\nStatement: 30% of the primary tr...
702,5c00f992-5066-42f2-88d7-566b8b9104b5,Single,Results,NCT00089661,NaN,On average cohort 1 participants in the primar...,Contradiction,Outcome Measurement: Lumbar Spine Bone Mine...,NaN,Outcome Measurement: Lumbar Spine Bone Mine...,Label: Contradiction\nPrimary Evidence Indexes...,Type: Single\nStatement: On average cohort 1 p...
703,4c0ffc50-5cb2-4574-b954-335d142edbe4,Single,Eligibility,NCT01527487,NaN,Patients prescribed Citalopram as an ongoing t...,Entailment,Exclusion Criteria: Chronic use of drugs tha...,NaN,Inclusion Criteria: Histologically confirmed...,Label: Entailment\nPrimary Evidence Indexes = ...,Type: Single\nStatement: Patients prescribed C...
704,4e3379b9-f971-4ab6-8846-c395f1162bf2,Comparison,Adverse Events,NCT00810797,NCT00828074,Skin infections were more common in patients i...,Entailment,Adverse Events 1: Skin infection * 1/36 (2.78%),Adverse Events 1: Skin infection * 2/41 (4.88%),Adverse Events 1: Total: 4/36 (11.11%) Rec...,Label: Entailment\nPrimary Evidence Indexes = ...,Type: Comparison\nStatement: Skin infections w...
705,9872394c-18b7-4f88-849e-0e2831e6ebf0,Single,Eligibility,NCT00579826,NaN,Participants of the primary trial cannot be cu...,Contradiction,Inclusion Criteria: On a stable dose of horm...,NaN,Inclusion Criteria: Post-menopausal women at...,Label: Contradiction\nPrimary Evidence Indexes...,Type: Single\nStatement: Participants of the p...


In [26]:
model_id = "ft:gpt-3.5-turbo-0125:personal::9iyNIzGd"

In [27]:
df2['Input'][701]

'Type: Single\nStatement: 30% of the primary trial participants had an increased level of CECs after 42 days of treatment.\nReference Data: Outcome Measurement:    Fraction of Patients With Increased Levels of Circulating Endothelial Cells   An exact 95% confidence interval (CI) will be calculated for the CEC response rate. With 26 patients, this CI will be no wider than 40% (e.g., if 13 of 26 patients respond, the CI is 30% to 70%).   Time frame: After 3 weeks of treatment Results 1:    Arm/Group Title: Treatment (Cediranib Maleate)   Arm/Group Description: Patients receive oral AZD2171 once daily for 42 days. Courses repeat every 42 days in the absence of disease progression or unacceptable toxicity.   Overall Number of Participants Analyzed: 26   Measure Type: Number   Unit of Measure: percentage of participants  30'

In [34]:
completion = client.chat.completions.create(
    model = model_id,
    messages = [{"role":"system", "content": system_prompt},
    {"role":"user", "content": df2['Input'][701]}]
    )

In [35]:
print(completion.choices[0].message.content)

Label: Entailment
Primary Evidence Indexes = Outcome Measurement:    Fraction of Patients With Increased Levels of Circulating Endothelial Cells   An exact 95% confidence interval (CI) will be calculated for the CEC response rate. With 26 patients, this CI will be no wider than 40% (e.g., if 13 of 26 patients respond, the CI is 30% to 70%).   Time frame: After 3 weeks of treatment Results 1:    Arm/Group Title: Treatment (Cediranib Maleate)   Arm/Group Description: Patients receive oral AZD2171 once daily for 42 days. Courses repeat every 42 days in the absence of disease progression or unacceptable toxicity.   Overall Number of Participants Analyzed: 26   Measure Type: Number   Unit of Measure: percentage of participants  30
Secondary Evidence Indexes = nan



In [36]:
df2['Label'][701]

'Contradiction'

In [37]:
df2['Primary_evidence_index'][701]

'Outcome Measurement:    Fraction of Patients With Increased Levels of Circulating Endothelial Cells   An exact 95% confidence interval (CI) will be calculated for the CEC response rate. With 26 patients, this CI will be no wider than 40% (e.g., if 13 of 26 patients respond, the CI is 30% to 70%).   Time frame: After 3 weeks of treatment Results 1:    Arm/Group Title: Treatment (Cediranib Maleate)   Arm/Group Description: Patients receive oral AZD2171 once daily for 42 days. Courses repeat every 42 days in the absence of disease progression or unacceptable toxicity.   Overall Number of Participants Analyzed: 26   Measure Type: Number   Unit of Measure: percentage of participants  30'

In [38]:
df2['Secondary_evidence_index'][701]

nan

In [39]:
completion = client.chat.completions.create(
    model = model_id,
    messages = [{"role":"system", "content": system_prompt},
    {"role":"user", "content": df2['Input'][702]}]
    )

In [40]:
print(completion.choices[0].message.content)

Label: Contradiction
Primary Evidence Indexes = Outcome Measurement:    Lumbar Spine Bone Mineral Density Percent Change From Baseline at Month 12   Bone Mineral Density Assessed by Dual Energy X-Ray Absorptiometry.   Time frame: 12 months Results 1:    Arm/Group Title: Denosumab 60 mg Q6M   Arm/Group Description: [Not Specified]   Overall Number of Participants Analyzed: 123   Least Squares Mean (95% Confidence Interval)   Unit of Measure: Percent Change from Baseline  4.8        (4.3 to 5.4) Results 2:    Arm/Group Title: Placebo   Arm/Group Description: [Not Specified]   Overall Number of Participants Analyzed: 122   Least Squares Mean (95% Confidence Interval)   Unit of Measure: Percent Change from Baseline  -.7        (-1.3 to -.1)
Secondary Evidence Indexes = nan



In [41]:
df2['Label'][702]

'Contradiction'

In [42]:
df2['Primary_evidence_index'][702]

'Outcome Measurement:    Lumbar Spine Bone Mineral Density Percent Change From Baseline at Month 12   Bone Mineral Density Assessed by Dual Energy X-Ray Absorptiometry.   Time frame: 12 months Results 1:    Arm/Group Title: Denosumab 60 mg Q6M   Arm/Group Description: [Not Specified]   Overall Number of Participants Analyzed: 123   Least Squares Mean (95% Confidence Interval)   Unit of Measure: Percent Change from Baseline  4.8        (4.3 to 5.4) Results 2:    Arm/Group Title: Placebo   Arm/Group Description: [Not Specified]   Overall Number of Participants Analyzed: 122   Least Squares Mean (95% Confidence Interval)   Unit of Measure: Percent Change from Baseline  -.7        (-1.3 to -.1)'

In [43]:
outputs = []


In [45]:
for i in range(701, 852):
    input_text = df2['Input'][idx]
    completion = client.chat.completions.create(
        model=model_id,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": df2['Input'][i]}
        ]
    )
    output_text = completion.choices[0].message.content
    outputs.append(output_text)

In [46]:
outputs[0]

'Label: Entailment\nPrimary Evidence Indexes = Outcome Measurement:    Fraction of Patients With Increased Levels of Circulating Endothelial Cells   An exact 95% confidence interval (CI) will be calculated for the CEC response rate. With 26 patients, this CI will be no wider than 40% (e.g., if 13 of 26 patients respond, the CI is 30% to 70%).   Time frame: After 3 weeks of treatment Results 1:    Arm/Group Title: Treatment (Cediranib Maleate)   Arm/Group Description: Patients receive oral AZD2171 once daily for 42 days. Courses repeat every 42 days in the absence of disease progression or unacceptable toxicity.   Overall Number of Participants Analyzed: 26   Measure Type: Number   Unit of Measure: percentage of participants  30\nSecondary Evidence Indexes = nan\n'

In [47]:
for i in range(853, 900):
    input_text = df2['Input'][idx]
    completion = client.chat.completions.create(
        model=model_id,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": df2['Input'][i]}
        ]
    )
    output_text = completion.choices[0].message.content
    outputs.append(output_text)

In [48]:
processed_data = []

In [49]:
for output in outputs:
    lines = output.split('\n')
    label = lines[0].split(': ')[1]
    primary_evidence_indexes = lines[1].split('= ')[1]
    secondary_evidence_indexes = lines[2].split('= ')[1] if 'Secondary Evidence Indexes' in output else 'nan'

    processed_data.append({
        'Label': label,
        'Primary Evidence Indexes': primary_evidence_indexes,
        'Secondary Evidence Indexes': secondary_evidence_indexes
    })

In [50]:
processed_data[0]

{'Label': 'Entailment',
 'Primary Evidence Indexes': 'Outcome Measurement:    Fraction of Patients With Increased Levels of Circulating Endothelial Cells   An exact 95% confidence interval (CI) will be calculated for the CEC response rate. With 26 patients, this CI will be no wider than 40% (e.g., if 13 of 26 patients respond, the CI is 30% to 70%).   Time frame: After 3 weeks of treatment Results 1:    Arm/Group Title: Treatment (Cediranib Maleate)   Arm/Group Description: Patients receive oral AZD2171 once daily for 42 days. Courses repeat every 42 days in the absence of disease progression or unacceptable toxicity.   Overall Number of Participants Analyzed: 26   Measure Type: Number   Unit of Measure: percentage of participants  30',
 'Secondary Evidence Indexes': 'nan'}

In [28]:
output_df = pd.read_csv('output.csv')

In [29]:
output_df.head()

,Label,Primary Evidence Indexes,Secondary Evidence Indexes
0,Entailment,Outcome Measurement: Fraction of Patients W...,NaN
1,Contradiction,Outcome Measurement: Lumbar Spine Bone Mine...,NaN
2,Entailment,Exclusion Criteria: Concurrent treatment wit...,NaN
3,Contradiction,Adverse Events 1: Skin infection * 1/36 (2.78%),Adverse Events 1: Skin infection * 2/41 (4.88%)
4,Entailment,Inclusion Criteria: Post-menopausal women at...,NaN


In [53]:
output_df.to_csv('output.csv', index=False)

In [30]:
df_final = pd.read_csv('merged_dataset_new.csv')

In [31]:
df_final['Response_strings']= df_final.apply(lambda row: f"""Label: {row['Label']}
Primary Evidence Indexes = {row['Primary_evidence_index']}
Secondary Evidence Indexes = {row['Secondary_evidence_index']}
""", axis = 1)

In [32]:
df_final['Input'] = "Type: "+ df_final['Type']+ "\nStatement: "+ df_final['Statement']+ "\nReference Data: " + df_final['Reference_data']

In [33]:
df_final.to_csv('final_dataset.csv', index=False)

In [34]:
df.columns


Index(['test_ID', 'Type', 'Section_id', 'Primary_id', 'Secondary_id',
       'Statement', 'Label', 'Primary_evidence_index',
       'Secondary_evidence_index', 'Reference_data', 'Response_strings',
       'Input'],
      dtype='object')

In [145]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

df_subset = df_final.iloc[701:898]

df_subset['Label'] = df_subset['Label'].astype(str).str.strip()
output_df['Label'] = output_df['Label'].astype(str).str.strip()

df_subset['Primary_evidence_index'] = df_subset['Primary_evidence_index'].astype(str).str.strip()
output_df['Primary Evidence Indexes'] = output_df['Primary Evidence Indexes'].astype(str).str.strip()

df_subset['Secondary_evidence_index'] = df_subset['Secondary_evidence_index'].astype(str).str.strip()
output_df['Secondary Evidence Indexes'] = output_df['Secondary Evidence Indexes'].astype(str).str.strip()

valid_labels = ['Entailment', 'Contradiction']
df_subset = df_subset[df_subset['Label'].isin(valid_labels)]
output_df = output_df[output_df['Label'].isin(valid_labels)]

label_accuracy = accuracy_score(df_subset['Label'], output_df['Label'])
label_f1 = f1_score(df_subset['Label'], output_df['Label'], average='weighted')
label_confusion_matrix = confusion_matrix(df_subset['Label'], output_df['Label'], labels=valid_labels)

def normalize_whitespace(text):
    return ' '.join(text.split()) if isinstance(text, str) else text

correct_primary_matches = 0
total_primary_length = 0
correct_secondary_matches = 0
total_secondary_length = 0

df_subset = df_final.iloc[701:898].copy()
output_df_subset = output_df.iloc[0:198].copy()

df_subset['Primary_evidence_index'] = df_subset['Primary_evidence_index'].astype(str).str.strip().apply(normalize_whitespace)
output_df_subset['Primary Evidence Indexes'] = output_df_subset['Primary Evidence Indexes'].astype(str).str.strip().apply(normalize_whitespace)

df_subset['Secondary_evidence_index'] = df_subset['Secondary_evidence_index'].astype(str).str.strip().apply(normalize_whitespace)
output_df_subset['Secondary Evidence Indexes'] = output_df_subset['Secondary Evidence Indexes'].astype(str).str.strip().apply(normalize_whitespace)

for i in range(len(output_df_subset)):
   
    primary_true_set = set(df_subset['Primary_evidence_index'].iloc[i].split('.'))
    primary_predicted_set = set(output_df_subset['Primary Evidence Indexes'].iloc[i].split('.'))

    secondary_true_set = set(df_subset['Secondary_evidence_index'].iloc[i].split('.'))
    secondary_predicted_set = set(output_df_subset['Secondary Evidence Indexes'].iloc[i].split('.'))

    primary_matches = len(primary_true_set.intersection(primary_predicted_set))
    correct_primary_matches += primary_matches

    total_primary_length += len(primary_true_set)

    secondary_matches = len(secondary_true_set.intersection(secondary_predicted_set))
    correct_secondary_matches += secondary_matches

    total_secondary_length += len(secondary_true_set)

primary_evidence_accuracy = correct_primary_matches / total_primary_length if total_primary_length > 0 else 0
secondary_evidence_accuracy = correct_secondary_matches / total_secondary_length if total_secondary_length > 0 else 0

print(f"Label Accuracy: {label_accuracy}")
print(f"Label F1 Score: {label_f1}")
print("Label Confusion Matrix:")
print(label_confusion_matrix)

print(f"Primary Evidence Index Accuracy: {primary_evidence_accuracy:.2f}")
print(f"Secondary Evidence Index Accuracy: {secondary_evidence_accuracy:.2f}")

Label Accuracy: 0.5126903553299492
Label F1 Score: 0.5143298546520794
Label Confusion Matrix:
[[57 53]
 [43 44]]
Primary Evidence Index Accuracy: 0.19
Secondary Evidence Index Accuracy: 0.37


<ipython-input-145-a0f746c011df>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['Label'] = df_subset['Label'].astype(str).str.strip()
<ipython-input-145-a0f746c011df>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['Primary_evidence_index'] = df_subset['Primary_evidence_index'].astype(str).str.strip()
<ipython-input-145-a0f746c011df>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead
